In [15]:
### inputs ###
task = "bart" # rest or bart
atLas = "mmp" # msdl or haox

In [16]:
import os
import scipy.stats
import numpy as np
from numpy import genfromtxt
import astropy.table as table
import astropy.units as u
import sys
from astropy.table import Table, Column, MaskedColumn
from astropy.io import ascii 
import openpyxl
import csv

In [17]:
# save matrix in a vector, removing its diagonal
def reshape_no_diag(input_matrix):
    L = np.size(input_matrix, axis = 0)
    H_vec = np.reshape(input_matrix, (1, L*L))
    for i in reversed(xrange(L)):
        H_vec = np.delete(H_vec, i*L+i)
    return H_vec

In [18]:
## reading the subject's ID's and groups
wb = openpyxl.load_workbook('/Users/pariarezaeinia/Dropbox/codes/Net-Neuro/Subjects1.xlsx')
wb.get_sheet_names()
sheet = wb.get_sheet_by_name('Sheet1')
N = sheet.max_row

In [19]:
## initializing the vectors containing subject's information and hitting time statistics
Name_vec = []
task_vec = []
group_vec = []
Age = []
Gender = []
mean_vec = np.zeros((N-1, 1))
var_vec = np.zeros((N-1, 1))
skew_vec = np.zeros((N-1, 1))
kurtosis_vec = np.zeros((N-1, 1))
median_vec = np.zeros((N-1, 1))
kelly_vec = np.zeros((N-1, 1))
yule_vec = np.zeros((N-1, 1))
Q10_vec = np.zeros((N-1, 1))
Q25_vec = np.zeros((N-1, 1))
Q75_vec = np.zeros((N-1, 1))
Q90_vec = np.zeros((N-1, 1))
Q90_10_vec = np.zeros((N-1, 1))
Q75_25_vec = np.zeros((N-1, 1))

In [20]:
## reading hitting time matrices and calculating the statistics
Data_DIR = "/Volumes/Seagate/UCLA/func/"
os.chdir(Data_DIR)
for i in range(1,N):
    ID = sheet.cell(row=i+1, column=1).value
    Subject_grp = sheet.cell(row=i+1, column=2).value
    Subject_age = sheet.cell(row=i+1, column=3).value
    Subject_gender = sheet.cell(row=i+1, column=4).value
    sub = str(ID) 
    Data_Name = "sub-" + sub
    Name_vec.append(Data_Name)
    Data_Name = "sub-" + sub + "_task-" + task + "_bold_space-MNI152NLin2009cAsym_preproc"    
    H = genfromtxt("hit_time_" + atLas + "_data_filt_partial_fixed_thrshld_linear_normalization_" + Data_Name, delimiter=',')
    L = np.size(H,axis = 0)
    H_vec = reshape_no_diag(H)
    task_vec.append(task)
    group_vec.append(Subject_grp)
    Age.append(Subject_age)
    Gender.append(Subject_gender)
    mean_vec[i-1] = np.mean(H_vec)
    var_vec[i-1] = np.var(H_vec)
    H_skew = scipy.stats.skew(H_vec)
    H_kurtosis = scipy.stats.kurtosis(H_vec)
    skew_vec[i-1] = H_skew
    kurtosis_vec[i-1] = H_kurtosis
    Q10_vec[i-1] = np.percentile(H_vec,10)
    Q25_vec[i-1] = np.percentile(H_vec,25)
    Q75_vec[i-1] = np.percentile(H_vec,75)
    Q90_vec[i-1] = np.percentile(H_vec,90)
    Q90_10_vec[i-1] = Q90_vec[i-1] - Q10_vec[i-1]
    Q75_25_vec[i-1] = Q75_vec[i-1] - Q25_vec[i-1]
    median_vec[i-1] = np.median(H_vec)  
    kelly_vec[i-1] = Q90_vec[i-1] + Q10_vec[i-1] - 2*median_vec[i-1]
    yule_vec[i-1] = kelly_vec[i-1]/median_vec[i-1]

In [21]:
## saving output data containing subject's information and hitting time statistics
t = table.Table()
t.add_column(table.Column(name='Name', data=Name_vec))
t.add_column(table.Column(name='Task', data=task_vec))
t.add_column(table.Column(name='Group', data=group_vec))
t.add_column(table.Column(name='Age', data=Age))
t.add_column(table.Column(name='Gender', data=Gender))
t.add_column(table.Column(name='Mean', data=mean_vec))
t.add_column(table.Column(name='Skewness', data=skew_vec))
t.add_column(table.Column(name='Varianc', data=var_vec))
t.add_column(table.Column(name='Kurtosis', data=kurtosis_vec))
t.add_column(table.Column(name='Median', data=median_vec))
t.add_column(table.Column(name='kelly', data=kelly_vec))
t.add_column(table.Column(name='yule', data=yule_vec))
t.add_column(table.Column(name='Q10', data=Q10_vec))
t.add_column(table.Column(name='Q25', data=Q25_vec))
t.add_column(table.Column(name='Q75', data=Q75_vec))
t.add_column(table.Column(name='Q90', data=Q90_vec))
t.add_column(table.Column(name='Q90_10', data=Q90_10_vec))
t.add_column(table.Column(name='Q75_25', data=Q75_25_vec))

out_path = "/Users/pariarezaeinia/desktop/Data/stats_filt_partial_normalization_" + task + "_" + atLas + ".csv"

if os.path.exists(out_path):
    os.remove(out_path)
ascii.write(t, out_path, delimiter = ',') 